# Dianグループブリーフィング同報通信プログラム
このプログラムはDianグループの実際の状況に応えましてミラクル洋（或いは張中洋）より造られたものである。
何か問題にでもあったら、どうか気楽に私と連絡して下さい！
本人のメールアドレス：mirakuruyoo@gmail.com

In [194]:
import os  
import smtplib  
import mimetypes  
import pandas as pd
import codecs
import chardet
import string
import time
from email.MIMEMultipart import MIMEMultipart  
from email.MIMEBase import MIMEBase  
from email.MIMEText import MIMEText  
from email.MIMEAudio import MIMEAudio  
from email.MIMEImage import MIMEImage  
from email import encoders
from email.utils import parseaddr, formataddr
from email.header import Header
from email.Encoders import encode_base64
from email.utils import COMMASPACE

########################################################################################
##　この脚本を使用する前に以下の内容をどうかご記入すること：
count=206             #本期简报的期数
account_number=0   #这次发送简报最开始使用第几个邮箱(我是16)(从1开始)
donated=0       #这次发送简报是有捐赠版(1)还是无捐赠版(0)
tired=0       #いつか次のメールアドレスに交換するをコントロールする量
##　もし本日このメールアドレスを使用したことがございませんとしたら，tiredを０のままにして頂戴
##　でなければ、回数を５０で割る数字を記入してください
########################################################################################

In [224]:
md = pd.read_excel(u'./index/mydutypart.xlsx',header=None)#test_mails
emails = [x.encode('utf-8') for x in md[2] if pd.isnull(x)==False]
we = pd.read_excel(u'./index/work_emails.xlsx')
gmailUser = list(we[u'账号'])[account_number]
gmailPassword = list(we[u'密码'])[account_number]

In [225]:
print list(we[u'账号'])[account_number],list(we[u'密码'])[account_number]
# print md[2]

dian@hust.edu.cn Diannewsletter2017


In [226]:
if count%2==0:
    filepathprefix = u"./source/"+str(count)+u'/无捐款/'
else:
    filepathprefix = u"./source/"+str(count)+'/'
for fname in os.listdir(filepathprefix):
    if os.path.splitext(fname)[1] == '.pdf':
        pdf_name = fname
    elif os.path.splitext(fname)[1] == '.txt':
        txt_name = fname
pdf_path,txt_path = filepathprefix+pdf_name,filepathprefix+txt_name
subject = u"Dian团队工作简报第"+str(count)+u"期"

with open(txt_path, 'rb') as fp:
    file_data = fp.read()
    result = chardet.detect(file_data)
    content = file_data.decode(encoding=result['encoding'])

In [227]:
print result['encoding']

GB2312


In [232]:
def _format_addr(s):
    name, addr = parseaddr(s)
    return formataddr(( \
        Header(name, 'gb2312').encode(), \
        addr.encode('gb2312') if isinstance(addr, unicode) else addr))
 
def prepmsg(subject, text, *attachmentFilePaths):
    global msg
    recipient = []  
    # ルートコンテナをセットします
    msg = MIMEMultipart()  
    msg['From'] = _format_addr(u'Newsletter Dian <%s>' % gmailUser) 
    msg['To'] = COMMASPACE.join(recipient)
    msg['Subject'] = Header(subject, 'gb2312').encode()  
    msg.attach(MIMEText(content, 'plain', 'gb2312'))  
    for attachmentFilePath in attachmentFilePaths:  
        msg.attach(getAttachment(attachmentFilePath)) 
    
def sendMail(other):   
    global msg
    global mailServer
    recipient = []
    msg['Bcc'] = COMMASPACE.join(other)
    print '='*60,"\n現在では発信することを実行しているアカウントは：",gmailUser
    mailServer.sendmail(gmailUser, recipient+other, msg.as_string())  
    print "Sent email to ", other
   
def getAttachment(attachmentFilePath):  
    contentType, encoding = mimetypes.guess_type(attachmentFilePath)  
    if contentType is None or encoding is not None:  
        contentType = 'application/octet-stream' 
    mainType, subType = contentType.split('/', 1)  
    file = open(attachmentFilePath, 'rb')  
   
    if mainType == 'text':  
        attachment = MIMEText(file.read())  
    elif mainType == 'message':  
        attachment = email.message_from_file(file)  
    elif mainType == 'image':  
        attachment = MIMEImage(file.read(),_subType=subType)  
    elif mainType == 'audio':  
        attachment = MIMEAudio(file.read(),_subType=subType)  
    else:  
        attachment = MIMEBase(mainType, subType)  
    attachment.set_payload(file.read())  
    encode_base64(attachment)    
    file.close()  

    ## 添付ファイルのヘッダをセットします
    attachment.add_header('Content-Disposition', 'attachment',   filename=os.path.basename(attachmentFilePath))  
    return attachment  


def OneUsrSendMail():
    global tired
    global mailServer
    start = time.time()
    allsent=0
    rested=len(emails)
    mailServer = smtplib.SMTP('mail.hust.edu.cn',25)
    mailServer.ehlo()  
    mailServer.starttls()  
    mailServer.ehlo()  
    mailServer.login(gmailUser, gmailPassword)  
    prepmsg(subject, content, pdf_path)
    while(rested!=0):
        if(rested>=50):
            sendMail(emails[allsent:(allsent+50)])
            allsent=allsent+50
            print ("この 50 通のメールはこのメールアドレスより出します：%s ，今既に %d 通のメールを送っています，残りの部分は直ぐに出しますので，少々お待ちください...\n" % (gmailUser.encode('UTF-8'),allsent)).decode('UTF-8')
            rested=rested-50
        else:
            sendMail(emails[allsent:])
            allsent=len(emails)
            print ("この %d 通のメールはこのメールアドレスより出します：%s ，もはや全部の %d 通のメールを送りましたので，ご利用ありがとうございます:)" % ((allsent%50),gmailUser.encode('UTF-8'),allsent)).decode('UTF-8')
            mailServer.close()  
            rested=0
        print '='*60
        if(tired>=50):
            resetMail()
        else:
            tired=tired+1
    end = time.time()
    print "\nこの度、本プログラムの実行時間は合わせて： ",end-start," 秒です。\n",'='*60

def resetMail():
    global account_number
    global gmailUser
    global gmailPassword
    global tired
    global mailServer
    account_number=account_number+1
    gmailUser = list(we[u'账号'])[account_number]
    gmailPassword = list(we[u'密码'])[account_number]
    mailServer = smtplib.SMTP('mail.hust.edu.cn',25)#'smtp.gmail.com'　'smtp.qq.com' 587
    mailServer.ehlo()  
    mailServer.starttls()  
    mailServer.ehlo()  
    mailServer.login(gmailUser, gmailPassword)  
    tired=0

In [203]:
emails = ['786671043@qq.com']#,'dengpanstudio@outlook.com']
OneUsrSendMail()
tired=0
emails = [x.encode('utf-8') for x in md[2]]

現在では発信することを実行しているアカウントは： dian@hust.edu.cn
Sent email to  ['786671043@qq.com', 'dengpanstudio@outlook.com']
この 2 通のメールはこのメールアドレスより出します：dian@hust.edu.cn ，もはや全部の 2 通のメールを送りましたので，ご利用ありがとうございます:)


In [233]:
OneUsrSendMail()

現在では発信することを実行しているアカウントは： dian@hust.edu.cn
Sent email to  ['530180782@qq.com', '243678025@qq.com', '398018489@qq.com', '595064131@qq.com', '362483245@qq.com', '285340035@qq.com', '448280012@qq.com', '452779373@qq.com', '1043553227@qq.com', '554003572@qq.com', '1158658706@qq.com', '535603525@qq.com', '327526091@qq.com', '491163631@qq.com', '404127680@qq.com', '610363953@qq.com', '530413323@qq.com', '976581566@qq.com', '645366312@qq.com', '252297147@qq.com', '958715182@qq.com', '421184840@qq.com', '276140133@qq.com', '295642572@qq.com', '657227464@qq.com', '402236479@qq.com', '304470253@qq.com', '954881719@qq.com', '584720149@qq.com', '1113964742@qq.com', '541939857@qq.com', '463757313@qq.com', '634335492@qq.com', '593368912@qq.com', '849847679@qq.com', '672900644@qq.com', '779569651@qq.com', '406242205@qq.com', '735055817@qq.com', '836633828@qq.com', '476706564@qq.com', '625211428@qq.com', '172918923@qq.com', '359163787@qq.com', '119023118@qq.com', '292010344@qq.com', '516086546@qq.com',

Sent email to  ['heen_001@sina.com', 'linronghy@163.com', '1071263714@qq.com', 'webmaster@baidajob.com', 'sunchangqing1978@126.com', 'sheng1601@sohu.com', 'hengbingshiye@163.com', '4861312@sohu.com', '787776139@qq.com', 'ahchangjiangjc@alibaba.com.cn', 'taipu@nbtaipu.com', 'sznmx2008@tom.com', 'co000@000.com', 'jxgl@yahoo.cn', 'jdzjy@alibaba.com.cn', 'weianwl@163.com', 'bangxinweiye@163.com', 'mingchun168888@163.com', 'geli520@163.com', 'caiyin@mail.wzptt.zj.cn', 'NO@NO.COM', 'Hangtian@ahhtmt.com', 'wuyouxiang@111.com', 'cydisplay@163.com', 'tyjx315@126.com', 'jjfd@163.com', '7312158@163.com', 'wangzhi12@hotmail.com', 'hzqxfj@126.com', 'chinaexp@ccpit.org', 'key1993@tom.com', 'yutai88@163.com', 'hessp844@qq.com', 'linlvg@gxjx.cn', 'sccs888@126.com', 'gracefulxiao@163.com', 'bjxyti@126.com', 'shzheng@163.com', 'shxiangxiu@yahoo.com.cn', 'jinhuachilun@163.com', 'iecadmin@smesd.gov', 'atvseller@hotmail.com', 'baoxiuyu@yahoo.cn', 'qiu3443@yahoo.com.cn', '1111@sup-tek.com', 'info@zzdzjx.com

In [223]:
import time
start = time.time()
for i in range(10000):
    a=1
end = time.time()
print end-start

0.00153803825378
